## ReadMe
This is a Chemical Inventory and Hazard Analysis Pipeline. This script prompts the
user for Globally Harmonized System (GHS) H-codes; see here for reference:
https://pubchem.ncbi.nlm.nih.gov/ghs . There are 2 inputs. The first is a required Chemical Inventory or list stored as an .xlsx or .csv file. This file should have "Chemical_Inventory" in the file name and contain 2 required column names: "Chemical
Name" and "CAS Number". Please view the code block at the top of the pipeline. We recommend you run this code in Google Colab. Upload your chemical inventory to a Google Drive folder, making sure that the file path listed to the right of "source_folder" in the first code block of the pipeline code is correct. The second input is an optional folder containing PDF versions of laboratory protocols. Please store the protocols folder in the "source_folder" in your Google Drive ensuring that the name of the folder listed to the right of source_folder in the "protocols_folder" variable is correct.

We set "print_intermediate_steps" to false by default. Change to print_intermediate_steps = True if you want to print the chemical inventory at each step for debugging.

After uploading the chemical inventory, defining source_folder, protocols_folder, GHS H-codes and changing "print_intermediate_steps" if necessary, you can run the entire pipeline via the File menu at Runtime > Run all.

The inventory file is automatically loaded from your specified source folder, with missing CAS numbers populated using PubChem API. The pipeline retrieves GHS hazard classifications (H-codes), chemical name synonyms, and searches for protocol PDFs that mention these chemicals. The extracted hazard data is visualized through bar charts, Pareto analysis and lists to identify potential chemical hazards and their protocol associations.

The final outputs include processed chemical inventory lists, matched protocol hazards, and visual analytics, all saved as Excel files and PNG charts in "source_folder" for further analysis.

# Define source folder, chemical inventory and protocols folder names and locations

Chemical Inventory file is in .xlsx and two required columns of data - chemical names and cas # should be named 'Chemical Name' and 'CAS Number' respectively

In [ ]:
import os
import glob
from google.colab import drive
drive.mount('/content/drive')

# Define all person specific locations and file names
# Define source folder
source_folder = '/content/drive/My Drive/your source folder'

# Define location of Chemical Inventory, which is an input list including 'Chemical Names' and 'CAS Numbers' column names
inventory_files = glob.glob(os.path.join(source_folder, '*Chemical_Inventory*.xlsx'))

# Define protocol folder
protocols_folder = os.path.join(source_folder, 'your_protocols_folder')

# = True if you want to print intermediate steps of chemical inventory dataframe for debugging.
# Set at = False as default
print_intermediate_steps = False

# Prompt the user to input relevant GHS codes
print("Enter relevant GHS codes separated by commas (e.g., H200,H201,H360FD):")
relevant_ghs_codes = input().strip().split(',')

# Import packages and load Chemical Inventory

In [ ]:
# Import all necessary packages and load chemical inventory

!pip install thermo
!pip install bs4
!pip install pdfplumber
!pip install pubchempy

#import packages
import numpy as np
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from thermo.chemical import Chemical
import pubchempy as pcp
import tkinter
from tkinter import *
from tkinter import filedialog
import pdfplumber
import requests
import json
import matplotlib.pyplot as plt
from collections import Counter
import networkx as nx

df_inventory = pd.DataFrame()
# Load .xlsx or .csv Chemical Inventory file
try:
    if len(inventory_files) == 0:
        print("No .xlsx files found with 'Chemical Inventory' in the name. Searching for .csv files...")

        # Search for .csv files
        inventory_files = glob.glob(os.path.join(source_folder, '*Chemical_Inventory*.csv'))

        if len(inventory_files) == 0:
            print("No .csv files found either. Please check the source folder.")
        else:
            # Use the first .csv file that matches the pattern
            csv_path = inventory_files[0]
            print(f"Loading file: {csv_path}")
            # Load the .csv file into a DataFrame
            df_inventory = pd.read_csv(
                csv_path,
                usecols=["Chemical Name", "CAS Number"]  # Adjust columns to match your file
            )
            print("CSV file loaded successfully.")
    else:
        # Use the first .xlsx file that matches the pattern
        excel_path = inventory_files[0]
        print(f"Loading file: {excel_path}")
        # Load the .xlsx file into a DataFrame
        df_inventory = pd.read_excel(
            excel_path,
            engine='openpyxl',
            usecols=["Chemical Name", "CAS Number"]  # Adjust columns to match your file
        )
        print("Excel file loaded successfully.")

    # Display the first few rows of the DataFrame
    print(df_inventory.head())
except ValueError as ve:
    print(f"Error loading file: {ve}")
except FileNotFoundError as fnfe:
    print(f"File not found: {fnfe}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


#CAS Number filter
Chemicals with no CAS numbers are mostly proprietary reagents.




In [ ]:
print(f"....................Populating missing CAS Numbers")

def get_cas_number(chemical_name):
    try:
        # Construct the primary search URL for PubChem
        search_url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{chemical_name}/xrefs/RegistryID/JSON"
        response = requests.get(search_url)

        if response.status_code == 200:
            # Extract CAS Number from response content
            response_data = response.json()
            # Locate the CAS Number in the JSON response
            registry_ids = response_data.get("InformationList", {}).get("Information", [])
            for entry in registry_ids:
                if "CAS" in entry.get("RegistryID", ""):
                    return entry["RegistryID"]  # Return the CAS Number

        # If the first URL does not return a result, try the fallback URL
        fallback_url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/substance/name/{chemical_name}/xrefs/RegistryID/JSON"
        response = requests.get(fallback_url)

        if response.status_code == 200:
            # Extract CAS Number from the fallback response
            response_data = response.json()
            registry_ids = response_data.get("InformationList", {}).get("Information", [])
            for entry in registry_ids:
                if "CAS" in entry.get("RegistryID", ""):
                    return entry["RegistryID"]  # Return the CAS Number

        return None  # Return None if no result is found in both URLs

    except Exception as e:
        print(f"Error finding CAS Number for {chemical_name}: {e}")
        return None


for index, row in df_inventory.iterrows():
    cas_number_individ = str(row["CAS Number"]).strip()  # Convert to string and strip spaces

    # Remove "00:00:00" or other trailing text from the CAS Number if present
    cas_number_individ = cas_number_individ.split(" ")[0].strip()  # Take only the first part before any space

    # Define a set of invalid characters
    invalid_characters = set(",./?!@#$%^&*():;\"'")

    # Check if CAS Number contains alphabetical letters or :
    for index, cas_number_individ in df_inventory["CAS Number"].items():
    # Convert the CAS Number to a string, handle NaN values by treating them as empty strings
        cas_number_individ = str(cas_number_individ) if not pd.isna(cas_number_individ) else ""
        # Check if there are any letters or invalid symbols in the CAS Number
        if any(char.isalpha() or char in invalid_characters for char in cas_number_individ):
            df_inventory.at[index, "CAS Number"] = ""  # Clear the CAS Number cell

    # Ensure CAS Number follows the correct format (remove leading zero in the last segment)
    if "-" in cas_number_individ:
        parts = cas_number_individ.split("-")
        if len(parts) == 3:
            try:
                parts[2] = str(int(parts[2]))  # Remove leading zeros from the last segment
                cas_number_individ = "-".join(parts)
                df_inventory.at[index, "CAS Number"] = cas_number_individ
            except ValueError:
                print(f"Error processing CAS Number: {cas_number_individ}")
                df_inventory.at[index, "CAS Number"] = ""  # Clear the invalid CAS Number
                continue

    # Check if the CAS Number is missing (NaN, blank, or 0)
    if pd.isna(row["CAS Number"]) or row["CAS Number"] in ["", 0]:
        # Try to find the CAS Number using the function
        chemical_name = row["Chemical Name"]  # Extract the chemical name
        cas_number = get_cas_number(chemical_name)  # Fetch CAS Number

        # If a CAS Number is retrieved, update the DataFrame
        if cas_number:
            df_inventory.at[index, "CAS Number"] = cas_number



print(f"....................Extracting and saving list of Chemical Names with no CAS Number")

# Initialize a new DataFrame for proprietary or other chemicals
df_proprietaryRxs_andOther = pd.DataFrame(columns=["Chemical Name", "CAS Number"])

# Process each row to fill CAS Numbers or move to df_proprietaryRxs_andOther
for index, row in df_inventory.iterrows():
    if pd.isna(row["CAS Number"]) or row["CAS Number"] in ["", 0]:
        cas_number = get_cas_number(row["Chemical Name"])
        if cas_number:
            df_inventory.at[index, "CAS Number"] = cas_number
        else:
            # Add the row to df_proprietaryRxs_andOther
            df_proprietaryRxs_andOther = pd.concat(
                [df_proprietaryRxs_andOther, pd.DataFrame([row])],
                ignore_index=True
            )

# Remove rows with NaN, blank, or 0 in CAS Number from df_inventory
df_inventory = df_inventory[
    ~(df_inventory["CAS Number"].isnull() | (df_inventory["CAS Number"] == "") | (df_inventory["CAS Number"] == 0))
]

if print_intermediate_steps:
  # Save df_inventory to an Excel file
  output_filename = "df_inventory_withCAS.xlsx"
  output_path = os.path.join(source_folder, output_filename)
  df_inventory.to_excel(output_path, index=False)
  output_filename = None
  output_path = None

# Remove rows with non-unique Chemical Names, keeping first instance
df_proprietaryRxs_andOther["Chemical Name"] = df_proprietaryRxs_andOther["Chemical Name"].str.strip().str.lower()
df_proprietaryRxs_andOther = df_proprietaryRxs_andOther.drop_duplicates(subset="Chemical Name", keep="first")
df_proprietaryRxs_andOther["Chemical Name"] = df_proprietaryRxs_andOther["Chemical Name"].str.title()


# Save df_proprietaryRxs_andOther to an Excel file
output_filename = "Chemical_List_noCAS.xlsx"
output_path = os.path.join(source_folder, output_filename)
df_proprietaryRxs_andOther.to_excel(output_path, index=False)
output_filename = None
output_path = None

df_inventory_backup = df_inventory.copy()


#GHS H-Code Scraper
The code in the next two code blocks is minimally edited from github user chadr1989 at this source:

https://github.com/chadr1989/GHS-codes-from-CAS-numbers/blob/main/GHS_codes_NCBI_scrape.ipynb



In [ ]:
#  This cell will loop over the PubChem IDs and request GHS data from NCBI.  This could have a runtime of several minutes depending on your hardware and internet connection.
#  Chemicals will be matched to the PubChem compound database. If it is not found there try the substance database

result = requests.get(f'https://pubchem.ncbi.nlm.nih.gov/ghs/#_prec','lxml')
soup = BeautifulSoup(result.text,'lxml')

gross_precautions_list = list()
p_codes_list = list()
precaution_statements_list = list()

for i in soup.select('#pcode')[0].select('td'):
    gross_precautions_list.append(i.text)
for i in range(0,len(gross_precautions_list)):
    if not re.search(r'P\d\d\d',gross_precautions_list[i])==None:
        p_codes_list.append(gross_precautions_list[i])
for code in p_codes_list:
    precaution_statements_list.append(gross_precautions_list[gross_precautions_list.index(code)+1])
precaution_data_dict = {'P Codes':p_codes_list,'Precautionary Statements':precaution_statements_list}
df_precaution = pd.DataFrame(precaution_data_dict)
#  Note: The loaded spreadsheet must contain the CAS numbers of the chemicals in a column named "CAS"
#  This cell creates new columns in the dataframe and cross-references PubChem IDs with given CAS numbers

# Initialize new columns in df_inventory
df_inventory['PubChem ID'] = np.nan
df_inventory['GHS Codes'] = np.nan
df_inventory['Precautionary Statements'] = np.nan

# Loop through each CAS number in the DataFrame and get PubChem ID (CID) either from compound database or substance database
for i in df_inventory['CAS Number']:
    try:
        # Attempt to find PubChem ID using thermo.chemical.Chemical
        chem = Chemical(f'{i}')
        if chem.PubChem:
            df_inventory.loc[df_inventory['CAS Number'] == i, 'PubChem ID'] = chem.PubChem
        else:
            raise ValueError("No PubChem ID found via thermo.chemical.Chemical")
    except Exception:
        try:
            # First attempt: Use PubChem Compound API
            compound_url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{i}/cids/JSON"

            response = requests.get(compound_url)
            response.raise_for_status()  # Raise an error for HTTP issues

            # Parse JSON response for compound
            data = response.json()

            if 'IdentifierList' in data and 'CID' in data['IdentifierList']:
                cid = data['IdentifierList']['CID'][0]  # Get the first CID
                df_inventory.loc[df_inventory['CAS Number'] == i, 'PubChem ID'] = cid
                print(f"CID {cid} successfully added to 'PubChem ID' for CAS Number: {i}")  # Only debugging line retained
            else:
                raise ValueError("No CID found in Compound API response")

        except Exception:
            try:
                # Fallback: Use PubChem Substance API
                substance_url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/substance/name/{i}/cids/JSON"

                response = requests.get(substance_url)
                response.raise_for_status()  # Raise an error for HTTP issues

                # Parse JSON response for substance
                data = response.json()

                if 'InformationList' in data and 'Information' in data['InformationList']:
                    # Extract CID from the first match in the substance response
                    cids = [
                        cid
                        for info in data['InformationList']['Information']
                        if 'CID' in info
                        for cid in info['CID']
                    ]
                    if cids:
                        df_inventory.loc[df_inventory['CAS Number'] == i, 'PubChem ID'] = cids[0]
                        print(f"CID {cids[0]} successfully added to 'PubChem ID' using substance database for CAS Number: {i}")  # Only debugging line retained
                    else:
                        raise ValueError("No CID found in Substance API response")
                else:
                    raise ValueError("No information found in Substance API response")
            except Exception:
                pass  # Fail silently for any unhandled errors


In [ ]:
for chem_id in set(df_inventory['PubChem ID'].dropna()):
    result = requests.get(f'https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/{int(chem_id)}/JSON/?response_type=display&heading=GHS%20Classification','lxml')
    soup = BeautifulSoup(result.text,'lxml').text
    if len(soup) > 90:
        pattern_hits = [m.start() for m in re.finditer(r'"H\d\d\d', soup)]
        ghs_codes_set = set()
        for i in range(0,len(pattern_hits)):
            phrase_start = pattern_hits[i]
            phrase_end = soup[phrase_start+1:].find('"')
            ghs_codes_set.add(soup[phrase_start+1:phrase_start+5])
        ghs_codes_list = list()
        for c in ghs_codes_set:
            ghs_start = soup.find(c)
            ghs_end = soup[soup.find(c)+1:].find('"')+1
            ghs_codes_list.append(soup[ghs_start:ghs_start+ghs_end])
        joined_ghs = ' --- '.join(ghs_codes_list)
        df_inventory.loc[df_inventory['PubChem ID']==chem_id,'GHS Codes'] = joined_ghs
        p_list = list()
        for i in set(soup[soup.find('Precautionary Statement Codes'):].replace('and ','').replace(' ','').replace('"\n}','').split(',')):
            if i in p_codes_list:
                p_list.append(i+' '+df_precaution['Precautionary Statements'][df_precaution[df_precaution['P Codes']==i].index[0]])
        joined_p = ' --- '.join(p_list)
        df_inventory.loc[df_inventory['PubChem ID']==chem_id,'Precautionary Statements'] = joined_p
    else:
        pass

# Remove precautionary statements and show dataframe. These fail to extract from pubchem. Retain the code because the extraction fails without it.
df_inventory = df_inventory.iloc[:, :-1]
df_inventory.head()


In [ ]:
# Function to fetch GHS codes from PubChem API if this didn't work with compounds and Pubchem ID (most likely the chemical name is missing a PubchemID)
def fetch_ghs_code(chemical_name):
    try:
        response = requests.get(f'https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/substance/{int(chemical_name)}/JSON/?response_type=display&heading=GHS%20Classification','lxml')
        soup = BeautifulSoup(result.text,'lxml').text
        if response.status_code == 200:
            data = response.json()
            # Extracting GHS Codes, placeholder logic (adjust based on actual API structure)
            sections = data.get("Record", {}).get("Section", [])
            for section in sections:
                if "GHS Classification" in section.get("TOCHeading", ""):
                    return section.get("Information", [{}])[0].get("StringValue", "No GHS Codes Found")
            return "No GHS Codes Found"
        else:
            return f"Error {response.status_code}: Unable to fetch data"
    except Exception as e:
        return f"Error: {e}"

# Loop through each row and update blank GHS Codes
for index, row in df_inventory.iterrows():
    if not row["GHS Codes"]:  # Check if GHS Codes is blank
        chemical_name = row["Chemical Name"]
        ghs_code = fetch_ghs_code(chemical_name)
        df_inventory.at[index, "GHS Codes"] = ghs_code


In [ ]:
# Save the df_inventory with GHS codes DataFrame to an Excel file
if print_intermediate_steps:
  output_path = None
  output_filename = None
  output_filename = 'df_inventory_withGHScodes.xlsx'
  output_path = os.path.join(source_folder, output_filename)
  df_inventory.to_excel(output_path, index=False)
  output_path = None
  output_filename = None

#GHS H-Code filter

In [ ]:
# Initialize DataFrames for separating relevant and other GHS codes
relevant_ghs_df = pd.DataFrame(columns=df_inventory.columns)
other_ghs_df = pd.DataFrame(columns=df_inventory.columns)

# Regex pattern to find H### codes
pattern = r'H\d{3}[A-Z]*'

# Loop through each row in df_inventory and check the GHS Codes column
for index, row in df_inventory.iterrows():
    # Extract all H### codes from the GHS Codes cell
    ghs_codes = re.findall(pattern, str(row['GHS Codes'])) if not pd.isna(row['GHS Codes']) else []

    # Check if any relevant GHS code exists in this row
    if any(code in relevant_ghs_codes for code in ghs_codes):
        relevant_ghs_df = pd.concat([relevant_ghs_df, row.to_frame().T], ignore_index=True)
    else:
        other_ghs_df = pd.concat([other_ghs_df, row.to_frame().T], ignore_index=True)

# Display the first few rows of each list for confirmation
print("\nRelevant GHS Codes DataFrame:")
print(relevant_ghs_df.head())
print("\nIrrelevant GHS Codes DataFrame:")
print(other_ghs_df.head())


In [ ]:
# Save dataframe with all chemicals in inventory with relevant GHS codes to an Excel file
output_filename = None
output_path = None
output_filename = 'All_Inventory_Chemicals_with_relevantGHScodes.xlsx'
output_path = os.path.join(source_folder, output_filename)
relevant_ghs_df.to_excel(output_path, index=False)

if print_intermediate_steps:
  output_filename = None
  output_path = None
  output_filename = 'All_Inventory_Chemicals_with_irrelevantGHScodes.xlsx'
  output_path = os.path.join(source_folder, output_filename)
  other_ghs_df.to_excel(output_path, index=False)

#Unique Chemical Name filter & Compiler of in-list synonyms for unique CAS Numbers

In [ ]:
#replace df_inventory (all GHS codes) with relevant_ghs_df
df_inventory = relevant_ghs_df

# Convert CAS Number column to string type to avoid type comparison issues
df_inventory['CAS Number'] = df_inventory['CAS Number'].astype(str)

# Sort the dataframe based on 'CAS Number' in chronological order
df_sorted = df_inventory.sort_values(by='CAS Number')

# Create a new dataframe to store unique CAS Numbers
unique_cas_nums = pd.DataFrame(columns=df_inventory.columns)

# List to collect data for the new chemical name columns
new_columns_data = []

# Loop through the sorted dataframe to extract first entry for each unique CAS Number
for cas_number, group in df_sorted.groupby('CAS Number'):
    # Get the first row for the unique CAS Number
    first_entry = group.iloc[0].to_frame().T

    # Use pd.concat to append the first entry to unique_cas_nums
    unique_cas_nums = pd.concat([unique_cas_nums, first_entry], ignore_index=True)

    # Create a temporary list of Chemical Names matching the unique CAS Number
    chemical_names = group['Chemical Name'].tolist()

    # Remove duplicates from the chemical names list
    unique_chemical_names = list(dict.fromkeys(chemical_names))

    # Add the unique chemical names as a list (will be used to create new columns)
    new_columns_data.append(unique_chemical_names)

# Ensure new_columns_data is not empty before applying max()
if not new_columns_data or all(len(names) == 0 for names in new_columns_data):
    print("You have no chemicals in this inventory with relevant GHS codes!")
else:
    # Create a new dataframe for the chemical names, handling varying numbers of chemical names per CAS Number
    max_names = max([len(names) for names in new_columns_data])
    chemical_names_df = pd.DataFrame([
        names + [None] * (max_names - len(names)) for names in new_columns_data
    ], columns=[f'inventory synonym {i+1}' for i in range(max_names)])

    # Concatenate the chemical names dataframe with unique_cas_nums
    unique_cas_nums = pd.concat([unique_cas_nums.reset_index(drop=True), chemical_names_df.reset_index(drop=True)], axis=1)

In [ ]:
# Save the df_inventory with unique GHS codes DataFrame to an Excel file
if print_intermediate_steps:
  output_filename = None
  output_path = None
  output_filename = 'df_inventory_relevantGHScodes_uniquecodes_inlistsyns.xlsx'
  output_path = os.path.join(source_folder, output_filename)
  unique_cas_nums.to_excel(output_path, index=False)
  output_filename = None
  output_path = None

#PubChem Synonym requests using CIDs utilizing PUG REST service
Try hazards as compounds (CIDs) first, then substances (SIDs).
<br> Resource websites:
- https://pubchem.ncbi.nlm.nih.gov/docs/pug-rest-tutorial#section=How-PUG-REST-Works <br>
- https://pubchem.ncbi.nlm.nih.gov/docs/pug-rest#section=Synonyms

Example url for request: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/vioxx/synonyms/TXT
<br> Only change CID (in example: vioxx)

In [ ]:
# Replace df_inventory with unique_cas_nums
df_inventory = unique_cas_nums

# Define master list
master_list = df_inventory['CAS Number'].astype(str).str.strip().tolist()

def search_hazard_synonyms_and_save_to_excel(master_list, input_df):
    """
    Search for synonyms of CAS Numbers using the PubChem API and return the updated DataFrame.

    Parameters:
    master_list (list): List of CAS Numbers to search for.
    input_df (pd.DataFrame): The original DataFrame containing CAS Numbers and other information.

    Returns:
    pd.DataFrame: The merged DataFrame with synonyms.
    """
    results = []

    # Ensure the CAS Numbers in master_list are strings and stripped of spaces
    master_list = [str(cas_number).strip() for cas_number in master_list]

    for cas_number in master_list:
        print(f'Processing CAS Number: {cas_number}')  # Debugging step to verify CAS numbers

        try:
            # Create the URL for retrieving compound synonyms based on CAS Number
            url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{cas_number}/synonyms/TXT'

            # Make an HTTP GET request to the PubChem API
            response = requests.get(url)

            # Check if the request was successful
            if response.status_code == 200:
                synonyms_text = response.text.strip().split("\n")
                #print(f'CAS Number: {cas_number}')
                #print(f'Synonyms: {synonyms_text}')
                # Append the CAS number and all synonyms to the results list
                results.append([cas_number] + synonyms_text)
            elif response.status_code == 404:
                substance_url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/substance/name/{cas_number}/synonyms/TXT'
                substance_response = requests.get(substance_url)

                if substance_response.status_code == 200:
                    synonyms_text = substance_response.text.strip().split("\n")
                    results.append([cas_number] + synonyms_text)
                else:
                    print(f'Substance not found or another error: {substance_response.status_code}')
                    results.append([cas_number, f'Substance Error: {substance_response.status_code}'])

        except Exception as e:
            print(f'Error processing CAS Number: {cas_number}')
            print(f'Error message: {str(e)}')
            results.append([cas_number, f'Error: {str(e)}'])

    # Convert the results to a DataFrame (expand columns dynamically based on the max number of synonyms)
    df_synonyms = pd.DataFrame(results)

    # Assign column names dynamically: first column as 'CAS Number' and rest as 'Synonym 1', 'Synonym 2', etc.
    max_synonyms = df_synonyms.shape[1] - 1  # The first column is 'CAS Number', the rest are synonyms
    df_synonyms.columns = ['CAS Number'] + [f'Synonym {i}' for i in range(1, max_synonyms + 1)]

    # Merge the synonyms DataFrame back with the original DataFrame
    output_df = pd.merge(input_df, df_synonyms, on="CAS Number", how="left")

    # Return the DataFrame for further processing or saving
    return output_df


# Call the function and get the output DataFrame
output_df = search_hazard_synonyms_and_save_to_excel(master_list, df_inventory)

# Print the header for debugging
print("Output DataFrame Header:")
print(output_df.columns)

# Save the output DataFrame to an Excel file
if print_intermediate_steps:
  output_path = None
  output_filename = None
  output_filename = 'df_inventory_relevantGHScodes_uniquecodes_inlistsyns_ncbisyns.xlsx'
  output_path = os.path.join(source_folder, output_filename)
  output_df.to_excel(output_path, index=False)
  print(f"Excel file saved to: {output_path}")


In [ ]:
# Replace df_inventory with output_df
df_inventory = output_df

print('Removing duplicate synonyms')
# Iterate through each row and dynamically determine the range from column E (index 4) to the last column
for index, row in df_inventory.iterrows():
    # Calculate the shape (number of elements) of the row before changes
    before_shape = len([val for val in row[4:].values if pd.notna(val)])

    for col in df_inventory.columns[4:df_inventory.shape[1]]:  # Dynamically select up to the last column
        if pd.notna(row[col]):  # Only process non-NaN cells
            # Split the cell content into a list of values, remove duplicates while maintaining order, and rejoin
            unique_values = list(dict.fromkeys(row[col].split(";")))
            df_inventory.at[index, col] = ";".join(unique_values)

    # Calculate the shape (number of elements) of the row after changes
    after_shape = len([val for val in row[4:].values if pd.notna(val)])

dims=df_inventory.shape
print(dims)

output_path = None
output_filename = None
output_filename = 'df_inventory_relevantGHScodes_uniquecodes_inlistsyns_ncbisyns.xlsx'
output_path = os.path.join(source_folder, output_filename)
output_df.to_excel(output_path, index=False)
print(f"Excel file saved to: {output_path}")

# Pdfplumber dependent protocol search, match and record

In [ ]:
# Populate protocols with the names of PDF files
protocols = []
for filename in os.listdir(protocols_folder):
    if filename.endswith('.pdf'):
        protocols.append(filename[:-4])
print(protocols)

In [ ]:
# This is the protocol pdf matching code. Toward completeness, we include all in list and PubChem extracted synonyms for each CAS number
# This results in occasional false positives particuarly when PubChem synonym list contains commonly used words like 'Clean' or 'not have'
# The code outputs a hazards detail list so you can view the synonym found in the protocol pdf file and ignore the false positives.

# Initialize the master list and hazards list
master_list = []
hazards = []

# Populate the master list with CAS Numbers, synonyms, PubChem ID, and GHS Codes
for index, row in df_inventory.iterrows():
    cas_number = row['CAS Number']
    pubchem_id = row['PubChem ID']
    ghs_codes = row['GHS Codes']
    synonyms = row[4:]  # Get all synonyms starting from the 5th column

    for synonym in synonyms:
        if pd.notna(synonym):  # Ensure synonym is not NaN
            # Append a tuple with CAS number, synonym, PubChem ID, and GHS Codes
            master_list.append((cas_number, synonym.strip(), pubchem_id, ghs_codes))

# Convert master_list to a DataFrame for easier handling
master_list_df = pd.DataFrame(master_list, columns=['CAS Number', 'Synonym', 'PubChem ID', 'GHS Codes'])
master_list_df.columns = master_list_df.columns.str.replace(' ', '_')

# Display the resulting DataFrame
print(master_list_df)
master_list_df.shape

In [ ]:
# This is the code block where the pdf matching takes place. Code execution will take at least 2 minutes to run - with extensive lists it can take up to 3 hours to run.
# Initialize the hazards list and the matched details list
hazards = []
matched_details = []

# Iterate through the files in the protocols folder
for filename in os.listdir(protocols_folder):
    print(f"Processing protocol: {filename}")  # Current file being processed
    if filename.endswith('.pdf') and filename != "Hazards In Protocols.txt":
        extracted_text = ''  # Initialize extracted text for the current PDF
        try:
            with pdfplumber.open(os.path.join(protocols_folder, filename)) as pdf:
                for page_num, page in enumerate(pdf.pages):
                    page_text = page.extract_text() or ''  # Extract text from each page
                    extracted_text += page_text  # Append the page's text

        except Exception as e:
            print(f"Error processing protocol {filename}: {e}")  # Handle PDF processing errors
            continue

        # Search for matches in the extracted text across all synonyms
        matched_cas_numbers = []
        matched_pubchem_ids = []
        matched_ghs_codes = []

        for row in master_list_df.itertuples(index=True):
            cas_number = row.CAS_Number
            synonym = row.Synonym
            pubchem_id = row.PubChem_ID
            ghs_codes = row.GHS_Codes

            # Perform case-sensitive whole-word matching using regex
            pattern = fr'\b{re.escape(synonym)}\b'
            if re.search(pattern, extracted_text):  # Check for whole-word match
                matched_cas_numbers.append(cas_number)
                matched_pubchem_ids.append(pubchem_id)
                matched_ghs_codes.append(ghs_codes)

                # Append details for the new DataFrame
                matched_details.append({
                    'Protocol': filename,
                    'Synonym': synonym,
                    'CAS Number': cas_number,
                    'PubChem_ID': pubchem_id,
                    'GHS_Codes': ghs_codes
                })
        print(matched_cas_numbers)
        # Remove duplicate CAS numbers while keeping order
        unique_cas_set = set()
        unique_details = []
        unique_matched_cas_numbers = []
        unique_matched_pubchem_ids = []
        unique_matched_ghs_codes = []

        for i, cas in enumerate(matched_cas_numbers):
            if cas not in unique_cas_set:
                unique_cas_set.add(cas)
                unique_matched_cas_numbers.append(cas)
                unique_matched_pubchem_ids.append(matched_pubchem_ids[i])
                unique_matched_ghs_codes.append(matched_ghs_codes[i])
                unique_details.append(matched_details[i])

        # Replace the original lists with the unique versions
        matched_cas_numbers = unique_matched_cas_numbers
        matched_pubchem_ids = unique_matched_pubchem_ids
        matched_ghs_codes = unique_matched_ghs_codes
        matched_details = unique_details

        print(matched_cas_numbers)

        # Prepare hazard entry for each protocol
        list_name = filename.replace('.pdf', '')
        parts = list_name.split('_', 1)  # Split at the first underscore
        protocol = parts[0]
        source = parts[1] if len(parts) > 1 else ""

        if not matched_cas_numbers:
            matched_hazards_str = "N/A"
            print(f"No matches found for {filename}")
        else:
            matched_hazards_str = ', '.join(matched_cas_numbers)
            print(f"Matched CAS numbers for {filename}: {matched_hazards_str}")

        # Append protocol, source, and matched CAS numbers to hazards list
        hazards.append([protocol, source, matched_hazards_str])
        print(f"Added to hazards list: {protocol}, {source}, {matched_hazards_str}")

# Convert hazards list to a DataFrame
df_hazards = pd.DataFrame(hazards, columns=['Protocol', 'Source', 'Hazards'])

# Convert matched details list to a DataFrame
df_matched_details = pd.DataFrame(matched_details, columns=['Protocol', 'Specific word pulled from pdf', 'Matched CAS Number', 'Matched PubChem_ID', 'Matched GHS_Codes'])


In [ ]:
# Save the df_inventory with unique GHS codes DataFrame to an Excel file
output_filename = None
output_path = None
output_filename = 'hazards_in_protocols.xlsx'
output_path = os.path.join(source_folder, output_filename)
df_hazards.to_excel(output_path, index=False)
output_filename = None
output_path = None

# Save the df_inventory with unique GHS codes DataFrame to an Excel file
output_filename = None
output_path = None
output_filename = 'protocol_matched_hazard_details.xlsx'
output_path = os.path.join(source_folder, output_filename)
df_matched_details.to_excel(output_path, index=False)
output_filename = None
output_path = None

# Data visualizations

In [ ]:
df=df_hazards

# Count the number of hazards per protocol
df["Hazard Count"] = df["Hazards"].apply(lambda x: len(str(x).split(", ")) if pd.notna(x) else 0)

# Figure size
fig_size = (12, 6)  # 100%
font = {'family': 'DejaVu Sans', 'size': 14}

# Get top 10 protocols by hazard count
top_protocols = df.nlargest(10, "Hazard Count")[["Protocol", "Hazard Count"]]
print("Top 10 protocols with highest number of hazards:")
print(top_protocols)

# Plot bar chart showing the number of hazards per protocol
fig, ax = plt.subplots(figsize=fig_size)
ax.bar(df["Protocol"], df["Hazard Count"], color="blue")
ax.set_xlabel("Protocol", fontdict=font)
ax.set_ylabel("Number of Hazards", fontdict=font)
# Display protocol names on x-axis
ax.set_xticks(range(len(df["Protocol"])))
ax.set_xticklabels(df["Protocol"], rotation=45, ha="right", fontsize=8)
ax.set_title("Number of Hazards per Protocol", fontdict=font)

plt.show()
# Save bar chart to source folder
bar_chart_path = os.path.join(source_folder, "hazards_per_protocol.png")
fig.savefig(bar_chart_path)

# Extract and count CAS numbers
cas_list = df["Hazards"].dropna().str.split(", ").explode()
cas_counts = Counter(cas_list)

# Convert to DataFrame and rank by frequency
cas_df = pd.DataFrame(cas_counts.items(), columns=["CAS Number", "Frequency"]).sort_values(by="Frequency", ascending=False)

# Calculate cumulative percentage
cas_df["Cumulative Frequency"] = cas_df["Frequency"].cumsum()
cas_df["Cumulative Percentage"] = (cas_df["Cumulative Frequency"] / cas_df["Frequency"].sum()) * 100

# Determine Pareto threshold
threshold = 80
pareto_cutoff = cas_df[cas_df["Cumulative Percentage"] <= threshold]

# Plot Pareto chart
fig, ax1 = plt.subplots(figsize=fig_size)

# Bar plot for frequency
ax1.bar(cas_df["CAS Number"], cas_df["Frequency"], color="C0")
ax1.set_xlabel("Hazard", fontdict=font)
ax1.set_ylabel("Frequency", fontdict=font, color="C0")
ax1.tick_params(axis="y", labelcolor="C0")
#x.set_xticklabels(df["CAS Number"], rotation=45, ha="right", fontsize=8)

# Line plot for cumulative percentage
ax2 = ax1.twinx()
ax2.plot(cas_df["CAS Number"], cas_df["Cumulative Percentage"], color="C1", marker="o", linestyle="-")
ax2.axhline(y=threshold, color="gray", linestyle="--")
ax2.set_ylabel("Cumulative Percentage", fontdict=font, color="C1")
ax2.tick_params(axis="y", labelcolor="C1")

plt.title("Pareto Analysis of Hazard Frequency", fontdict=font)
plt.show()

print("CAS Numbers over 80% Pareto Threshold:")
print(pareto_cutoff["CAS Number"].tolist())
print("CAS Numbers all:")
print(cas_df["CAS Number"].tolist())

pareto_chart_path = os.path.join(source_folder, "pareto_analysis.png")
fig.savefig(pareto_chart_path)
plt.show()

